<a href="https://www.kaggle.com/code/stemosamaghandour/arc-prize-2025-by-stemer?scriptVersionId=252303806" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2025/sample_submission.json
/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json
/kaggle/input/egx-jul-2025/egx_jul_2025.xlsx


In [2]:
        data = pd.read_excel('/kaggle/input/egx-jul-2025/egx_jul_2025.xlsx', parse_dates=['Highest Price Date'], index_col='Highest Price Date')

In [3]:
print(data.columns)

Index(['Stock', 'Sector', 'Currenc y', 'Par Value', 'Shares', 'Paid Capital',
       'Open Price', 'Highest Price', 'Lowest Price', 'Lowest Price Date',
       'Close Price', 'YTD\nPrice Chg.\n%',
       'Open Price after avoiding Capital Structure change effect',
       'YTD Price Chg. % after avoiding Capital Structure change effect',
       'Market Capitalization',
       'Average daily value traded in EGP (No\nDeals)',
       'Average daily volume traded (No Deals)',
       'Average daily No. of Trades (No Deals)', 'Tradin g Days',
       'Last Annual Profit', 'Profit Date', 'P/E', 'MIN\nP/E', 'MAX\nP/E',
       'Sector P/E', 'Coupon', 'Payment Date', 'DY', 'Corporate Actions',
       'Index'],
      dtype='object')


In [4]:
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter, sobel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate

class ARCReasoner:
    def __init__(self, lookback_window=30, forecast_window=5, num_features=4):
        self.lookback_window = lookback_window
        self.forecast_window = forecast_window
        self.num_features = num_features
        self._build_model()
    
    def _build_model(self):
        """Build the neural network model"""
        # Chart data input
        chart_input = Input(shape=(self.lookback_window, self.num_features, 3), name='chart_input')
        
        # Numerical data input
        num_input = Input(shape=(self.num_features,), name='num_input')
        
        # Process chart data
        x = Conv2D(32, (3, 3), activation='relu')(chart_input)
        x = Flatten()(x)
        
        # Combine with numerical data
        merged = concatenate([x, num_input])
        
        # Output layer
        output = Dense(1)(merged)
        
        # Create and compile model
        self.model = Model(inputs=[chart_input, num_input], outputs=output)
        self.model.compile(optimizer='adam', loss='mse')
        print(f"Model built for {self.num_features} features")
    
    def _apply_transformations(self, window):
        """Apply transformations to window data"""
        try:
            if isinstance(window, pd.DataFrame):
                window = window.values.astype(np.float32)
            
            window = np.nan_to_num(window)
            smoothed = gaussian_filter(window, sigma=1)
            sobel_v = sobel(window, axis=0)
            sobel_h = sobel(window, axis=1)
            edges = np.sqrt(sobel_v**2 + sobel_h**2)
            
            return np.stack([window, smoothed, edges], axis=-1)
        except Exception as e:
            print(f"Transformation failed: {e}")
            return None
    
    def _prepare_data(self, stock_data):
        """Prepare training data"""
        # Select numeric data
        numeric_data = stock_data.select_dtypes(include=[np.number])
        if numeric_data.shape[1] < self.num_features:
            raise ValueError(f"Need at least {self.num_features} numeric features")
        
        # Use first num_features if too many
        if numeric_data.shape[1] > self.num_features:
            numeric_data = numeric_data.iloc[:, :self.num_features]
        
        numeric_data = numeric_data.ffill().bfill()
        
        chart_images = []
        numerical_data = []
        targets = []
        
        for i in range(self.lookback_window, len(numeric_data)-self.forecast_window):
            window = numeric_data.iloc[i-self.lookback_window:i]
            target = numeric_data.iloc[i:i+self.forecast_window].iloc[:, 0].mean()  # Using first column as target
            
            transformed = self._apply_transformations(window)
            if transformed is None:
                continue
                
            try:
                chart_images.append(transformed.reshape(
                    self.lookback_window,
                    self.num_features,
                    3
                ))
                numerical_data.append(window.mean().values)
                targets.append(target)
            except Exception as e:
                print(f"Window {i} processing failed: {e}")
                continue
        
        if not chart_images:
            raise ValueError("No valid windows created")
            
        return (np.array(chart_images), np.array(numerical_data)), np.array(targets)
    
    def train(self, stock_data, epochs=50, batch_size=32):
        """Train the model"""
        (X_chart, X_num), y = self._prepare_data(stock_data)
        
        print(f"Training on {len(X_chart)} samples")
        print(f"Shapes - X_chart: {X_chart.shape}, X_num: {X_num.shape}, y: {y.shape}")
        
        self.model.fit(
            [X_chart, X_num],
            y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            verbose=1
        )
    
    def predict(self, recent_data):
        """Make predictions"""
        (X_chart, X_num), _ = self._prepare_data(recent_data)
        return self.model.predict([X_chart, X_num])

2025-07-24 17:29:05.757742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753378146.038144      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753378146.120481      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
#Debugging Steps:
#Check Your Data:


print("Numeric columns:", data.select_dtypes(include=[np.number]).columns)
print("Number of features:", len(data.select_dtypes(include=[np.number]).columns))

Numeric columns: Index(['Par Value', 'Shares', 'Paid Capital', 'Open Price', 'Highest Price',
       'Lowest Price', 'Close Price', 'YTD\nPrice Chg.\n%',
       'Open Price after avoiding Capital Structure change effect',
       'YTD Price Chg. % after avoiding Capital Structure change effect',
       'Market Capitalization',
       'Average daily value traded in EGP (No\nDeals)',
       'Average daily volume traded (No Deals)',
       'Average daily No. of Trades (No Deals)', 'Tradin g Days',
       'Last Annual Profit', 'P/E', 'MIN\nP/E', 'MAX\nP/E', 'Sector P/E',
       'DY'],
      dtype='object')
Number of features: 21


In [6]:
import pandas as pd
import numpy as np

def clean_data(df):
    """Clean data while preserving datetime columns"""
    # Make a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Identify numeric and datetime columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    datetime_cols = df.select_dtypes(include=['datetime', 'datetime64']).columns
    
    # Process numeric columns
    if not numeric_cols.empty:
        # Replace infinities with NaN
        df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
        
        # Fill NaN values
        df[numeric_cols] = df[numeric_cols].ffill().bfill()
        
        # Clip extreme values only for numeric columns
        df[numeric_cols] = df[numeric_cols].clip(lower=-1e6, upper=1e6)
    
    # Process datetime columns (if any)
    if not datetime_cols.empty:
        # Forward fill datetime columns
        df[datetime_cols] = df[datetime_cols].ffill()
    
    return df

try:
    # 1. Load data
    data = pd.read_excel('/kaggle/input/egx-jul-2025/egx_jul_2025.xlsx')
    
    # 2. Clean data
    data = clean_data(data)
    
    # 3. Verify cleaning
    print("NaN values after cleaning:", data.isna().sum().sum())
    print("Data types:\n", data.dtypes)
    
    # 4. Continue with analysis
    num_features = len(data.select_dtypes(include=[np.number]).columns)
    print(f"Number of numeric features: {num_features}")
    
    # 5. Initialize and train model
    model = ARCReasoner(
        lookback_window=30,
        forecast_window=5,
        num_features=num_features
    )
    model.train(data)

except Exception as e:
    print(f"Error: {e}")
    raise

NaN values after cleaning: 512
Data types:
 Stock                                                                      object
Sector                                                                     object
Currenc y                                                                  object
Par Value                                                                 float64
Shares                                                                    float64
Paid Capital                                                              float64
Open Price                                                                float64
Highest Price                                                             float64
Highest Price Date                                                 datetime64[ns]
Lowest Price                                                              float64
Lowest Price Date                                                  datetime64[ns]
Close Price                                           

2025-07-24 17:29:21.389602: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Training on 203 samples
Shapes - X_chart: (203, 30, 21, 3), X_num: (203, 21), y: (203,)
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - loss: 9069860487168.0000 - val_loss: 4405584199680.0000
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2940620242944.0000 - val_loss: 2405146558464.0000
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1534663983104.0000 - val_loss: 2618481704960.0000
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1454451458048.0000 - val_loss: 435716784128.0000
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 729777176576.0000 - val_loss: 50451861504.0000
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 117043593216.0000 - val_loss: 294439780352.0000
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 98307629056.0000 - val_loss: 72090656768.0000
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 96367091712.0000 - val_loss: 116436271104.0000
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 45539512320

In [7]:
#Verify Model Input:


print("Model expects:", model.model.input_shape)


Model expects: [(None, 30, 21, 3), (None, 21)]


In [8]:
#Test Data Preparation:


(X_chart, X_num), y = model._prepare_data(data)
print("Prepared data shapes:", X_chart.shape, X_num.shape, y.shape)

Prepared data shapes: (203, 30, 21, 3) (203, 21) (203,)


whats app group : 
https://chat.whatsapp.com/LKmClYcwfwg4EHycDvxgox

** For winning  the ARC Prize 2025**, here’s a strategic **flowchart breakdown** tailored to your background in AI, programming, and education. This flowchart covers each major milestone from preparation to final submission, with a focus on **efficient novel reasoning system design**—not just model training.

---

### 🧠 **Winning Flowchart for ARC Prize 2025**

#### 🚀 Goal: Build an AI system that can reason and generalize like a human on novel ARC tasks.

---

### 🔍 **1. Understand the ARC Benchmark**

* 🔹 Study ARC-AGI-2 format and principles (Abstraction, Reasoning, Generalization).
* 🔹 Read previous winning approaches and ARC Prize 2024 papers.
* 🔹 Analyze sample input-output pairs carefully—focus on **how a human solves them**.
* 🛠 Tools: `matplotlib`, `json`, `numpy`

---

### 🧪 **2. Experiment with Baselines**

* 🔸 Start with the [ARC baseline repo](https://github.com/fchollet/ARC) or similar.
* 🔸 Test basic rule-based solvers or search-based approaches (DFS, A\* over grid ops).
* 🔸 Build interpretable models—try symbolic or modular systems.
* ⚠ Avoid overfitting to patterns seen in training tasks.

---

### 🔁 **3. Build & Iterate Your Reasoning Engine**

* ✅ Choose a core reasoning approach:

  * Rule learning / DSL induction
  * Program synthesis
  * Meta-learning or few-shot pattern recognition
* 🔄 Add modules:

  * Shape & color abstraction
  * Grid transformation engine
  * Multi-step reasoning memory
* 💡 Use human-interpretable debug outputs to analyze reasoning failures.

---

### 🧪 **4. Evaluate on Public Tasks**

* 📈 Use validation tasks to:

  * Score using the 2-attempt rule
  * Analyze where model fails vs. human
* 📂 Build `submission.json` in correct format
* 🧼 Ensure consistent task ordering

---

### 📊 **5. Optimize & Scale**

* 🔍 Improve speed and generalization:

  * Optimize data structures
  * Prune hypothesis trees
  * Use few-shot memory banks
* 🔧 Consider simple neural-symbolic hybrids—but avoid black-box LLMs only.

---

### 📝 **6. Prepare Final Submission**

* 🔐 Lock code for reproducibility
* 📄 Generate valid `submission.json`
* 📦 Optional: Dockerize or clean environment
* 🧠 Write a technical report (for Nov 9 paper award)

---

### 🗓️ **Key Dates**

| Milestone                    | Deadline         |
| ---------------------------- | ---------------- |
| Accept rules & join          | October 27, 2025 |
| Final code submission        | November 3, 2025 |
| Paper submission (for award) | November 9, 2025 |

---

### 🏆 **Mindset for Success**

* Think like a teacher: can your system **understand concepts** like you explain to students?
* Think like a puzzle-solver: is your system curious, exploratory, and patient?
* Think like an engineer: is your code **fast, modular, and clean**?



In [9]:



# File paths for ARC Prize 2025 (as per Kaggle input)
TEST_CHALLENGES_PATH = "/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json"
SUBMISSION_OUTPUT_PATH = "/kaggle/working/submission.json"

# 🔁 Dummy solver that just copies the input as output
# You should replace this with your real reasoning engine
def identity_solver(input_grid):
    return input_grid

# 🧠 Solver for a single task
def solve_task(task_data):
    solutions = []
    for test in task_data.get("test", []):
        inp = test["input"]
        solutions.append({
            "attempt_1": identity_solver(inp),
            "attempt_2": identity_solver(inp)
        })
    return solutions
# 🧾 Main function to generate a submission
def generate_submission():
    # Load test challenges
    with open(TEST_CHALLENGES_PATH, 'r') as f:
        test_tasks = json.load(f)

    submission = {}
    # Each task_id maps to its test cases
    for task_id, task_data in test_tasks.items():
        submission[task_id] = solve_task(task_data)

    # Save the output to submission.json
    with open(SUBMISSION_OUTPUT_PATH, 'w') as f:
        json.dump(submission, f)
    print(f"✅ Submission saved to {SUBMISSION_OUTPUT_PATH}")

# 🚀 Run
if __name__ == "__main__":
    generate_submission()

# Kaggle paths
DATA_DIR = '/kaggle/input/arc-prize-2025'  # Adjust to actual input name
OUTPUT_FILE = '/kaggle/working/submission.json'


# Dummy solver – replace this with actual logic
def identity_solver(input_grid):
    return input_grid

def solve_task(task):
    solutions = []
    for test_case in task['test']:
        inp = test_case['input']
        attempt1 = identity_solver(inp)
        attempt2 = identity_solver(inp)
        solutions.append({
            "attempt_1": attempt1,
            "attempt_2": attempt2
        })
    return solutions
def load_task(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

""" 
def generate_submission(test_dir):
    submission = {}
    for filename in os.listdir(test_dir):
        if filename.endswith('.json'):
            task_id = filename[:-5]
            path = os.path.join(test_dir, filename)
            task = load_task(path)
            submission[task_id] = solve_task(task)
    return submission
"""
def generate_submission():
    with open(TEST_CHALLENGES_PATH, 'r') as f:
        test_tasks = json.load(f)

    submission = {}
    for task_id, task_data in test_tasks.items():
        submission[task_id] = solve_task(task_data)
    
    return submission
    
def save_submission(submission, output_path):
    with open(output_path, 'w') as f:
        json.dump(submission, f)
    print(f'Saved: {output_path}')


"""  error 
# Execute all
if __name__ == '__main__':
    test_path = os.path.join(DATA_DIR)  # Expected structure: /test/*.json
    submission = generate_submission(test_path)
    save_submission(submission, OUTPUT_FILE)
    """

# Execute all
if __name__ == '__main__':
    submission = generate_submission()  # No path argument needed
    save_submission(submission, SUBMISSION_OUTPUT_PATH)



✅ Submission saved to /kaggle/working/submission.json
Saved: /kaggle/working/submission.json
